Combinar los datasets en uno solo mejora la eficiencia del análisis , y mantener una columna de origen para luego segmentar los resultados.

In [1]:
import pandas as pd

# Cargar los datasets con columna de origen
dataset='Android'
dir_path = "../data/"
android = pd.read_csv(f"{dir_path}{dataset}_2k.log_structured_to_process.csv")
android['source'] = dataset
android=android[['Content', 'Level', 'datetime', 'content_length', 'source']]
android.head(2)

,Content,Level,datetime,content_length,source
0,printFreezingDisplayLogsopening app wtoken = A...,Informativo,2024-03-17 16:13:38.811,21,Android
1,"acquire lock=233570404, flags=0x1, tag=""View L...",Informativo,2024-03-17 16:13:38.819,9,Android


In [2]:
dataset = 'Apache'
apache = pd.read_csv(f"{dir_path}{dataset}_2k.log_structured_to_process.csv")
apache['source'] = dataset
apache=apache[['Content', 'Level', 'datetime', 'content_length', 'source']]
apache.head(2)

,Content,Level,datetime,content_length,source
0,workerEnv.init() ok /etc/httpd/conf/workers2.p...,Informativo,2005-12-04 04:47:44.000,3,Apache
1,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:47:44.000,7,Apache


In [3]:
dataset = 'Hadoop'
hadoop = pd.read_csv(f"{dir_path}{dataset}_2k.log_structured_to_process.csv")
hadoop['source'] = dataset
hadoop=hadoop[['Content', 'Level', 'datetime', 'content_length', 'source']]
hadoop.head(2)

,Content,Level,datetime,content_length,source
0,Created MRAppMaster for application appattempt...,Informativo,2015-10-18 18:01:47.000,5,Hadoop
1,Executing with tokens:,Informativo,2015-10-18 18:01:48.000,3,Hadoop


In [4]:
dataset = 'Zookeeper'
zookeeper = pd.read_csv(f"{dir_path}{dataset}_2k.log_structured_to_process.csv")
zookeeper['source'] = dataset
zookeeper=zookeeper[['Content', 'Level', 'datetime', 'content_length', 'source']]
zookeeper.head(2)

,Content,Level,datetime,content_length,source
0,Notification time out: 3200,Informativo,2015-07-29 17:41:44.747,4,Zookeeper
1,Received connection request /10.10.34.11:45307,Informativo,2015-07-29 19:04:12.394,4,Zookeeper


In [5]:
dataset = 'BGL'
bgl = pd.read_csv(f"{dir_path}{dataset}_2k.log_structured_to_process.csv")
bgl['source'] = dataset
bgl=bgl[['Content', 'Level', 'datetime', 'content_length', 'source']]
bgl.head(2)

,Content,Level,datetime,content_length,source
0,instruction cache parity error corrected,Informativo,2005-06-03 22:42:50,5,BGL
1,instruction cache parity error corrected,Informativo,2005-06-03 22:42:53,5,BGL


In [6]:
# Unir todos los datasets en uno solo
df_all = pd.concat([android, apache, hadoop, zookeeper, bgl], ignore_index=True)
df_all.head()

,Content,Level,datetime,content_length,source
0,printFreezingDisplayLogsopening app wtoken = A...,Informativo,2024-03-17 16:13:38.811,21,Android
1,"acquire lock=233570404, flags=0x1, tag=""View L...",Informativo,2024-03-17 16:13:38.819,9,Android
2,"ready=true,policy=3,wakefulness=1,wksummary=0x...",Informativo,2024-03-17 16:13:38.820,1,Android
3,Skipping AppWindowToken{df0798e token=Token{78...,Informativo,2024-03-17 16:13:38.839,11,Android
4,visible is system.time.showampm,Informativo,2024-03-17 16:13:38.859,3,Android


In [37]:
df_all.to_csv(f"{dir_path}raw_data_log_structured.csv", index=False)

# Limpieza y estandarización

In [38]:
df_all.shape

(10000, 5)

In [39]:
#Remover duplicados
df_all.drop_duplicates(inplace=True)
df_all.dropna(subset=['Content', 'Level'], inplace=True)
df_all.shape

(9410, 5)

In [40]:
df_all.shape

(9410, 5)

In [41]:
df_all.head()

,Content,Level,datetime,content_length,source
0,printFreezingDisplayLogsopening app wtoken = A...,Informativo,2024-03-17 16:13:38.811,21,Android
1,"acquire lock=233570404, flags=0x1, tag=""View L...",Informativo,2024-03-17 16:13:38.819,9,Android
2,"ready=true,policy=3,wakefulness=1,wksummary=0x...",Informativo,2024-03-17 16:13:38.820,1,Android
3,Skipping AppWindowToken{df0798e token=Token{78...,Informativo,2024-03-17 16:13:38.839,11,Android
4,visible is system.time.showampm,Informativo,2024-03-17 16:13:38.859,3,Android


In [42]:
set(df_all['Level'])

{'Advertencia', 'Error', 'Informativo'}

 # Limpieza de texto y tokenización para NLP

In [43]:
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

stop_words = set(ENGLISH_STOP_WORDS)

def clean_text(text):
    text = str(text).lower().replace(",", " ").replace(".", " ").replace(":", " ").replace("=", " ")
    tokens = re.findall(r'\b[a-z]+\b', text)  # solo palabras alfabéticas
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)


In [44]:
test = "Hello, this455 is a test."
print(clean_text(test))


hello test


In [45]:
df_all['Clean_Content'] = df_all['Content'].apply(clean_text)
df_all['content_length'] = df_all['Clean_Content'].apply(lambda x: len(x.split()))

In [46]:
df_all.shape

(9410, 6)

In [47]:
df_all.head()

,Content,Level,datetime,content_length,source,Clean_Content
0,printFreezingDisplayLogsopening app wtoken = A...,Informativo,2024-03-17 16:13:38.811,22,Android,printfreezingdisplaylogsopening app wtoken app...
1,"acquire lock=233570404, flags=0x1, tag=""View L...",Informativo,2024-03-17 16:13:38.819,13,Android,acquire lock flags tag view lock com android s...
2,"ready=true,policy=3,wakefulness=1,wksummary=0x...",Informativo,2024-03-17 16:13:38.820,17,Android,ready true policy wakefulness wksummary uasumm...
3,Skipping AppWindowToken{df0798e token=Token{78...,Informativo,2024-03-17 16:13:38.839,17,Android,skipping appwindowtoken token token activityre...
4,visible is system.time.showampm,Informativo,2024-03-17 16:13:38.859,3,Android,visible time showampm


In [48]:
df_all.to_csv(f"{dir_path}clean_data_log_structured.csv", index=False)

# Estadísticas Descriptivas Generales


In [49]:
# Total de registros
total_registros = len(df_all)

# Conteo por nivel de severidad
niveles_totales = df_all['Level'].value_counts()




In [50]:
# Longitud promedio del contenido
longitud_promedio = df_all['content_length'].mean()

In [51]:
# Fechas mínimas y máximas
min_fecha = df_all['datetime'].min()
max_fecha = df_all['datetime'].max()
print(f"Periodo: desde {min_fecha} hasta {max_fecha}")

Periodo: desde 2005-06-03 22:42:50 hasta 2024-03-17 16:16:09.141


In [52]:
print(f"Total de registros: {total_registros}")
print(f"Distribución por nivel:\n{niveles_totales}")
print(f"Longitud promedio del mensaje: {longitud_promedio:.2f} palabras")

Total de registros: 9410
Distribución por nivel:
Level
Informativo    6176
Advertencia    2297
Error           937
Name: count, dtype: int64
Longitud promedio del mensaje: 5.35 palabras


**Por dataset**



In [83]:
# Agrupar por fuente
grouped = df_all.groupby('source')

# Recorrer cada grupo y calcular estadísticas
for name, group in grouped:
    print(f"\n Dataset: {name}")
    print(f"- Total registros: {len(group)}")
    print("- Distribución por nivel:")
    print(group['Level'].value_counts())
    print(f"- Longitud promedio del mensaje: {group['content_length'].mean():.2f} palabras")
    print(f"- Periodo: desde {group['datetime'].min()} hasta {group['datetime'].max()}")



 Dataset: Android
- Total registros: 1987
- Distribución por nivel:
Level
Informativo    1817
Advertencia     167
Error             3
Name: count, dtype: int64
- Longitud promedio del mensaje: 7.26 palabras
- Periodo: desde 2024-03-17 16:13:38.811 hasta 2024-03-17 16:16:09.141

 Dataset: Apache
- Total registros: 1461
- Distribución por nivel:
Level
Informativo    1083
Error           378
Name: count, dtype: int64
- Longitud promedio del mensaje: 3.84 palabras
- Periodo: desde 2005-12-04 04:47:44.000 hasta 2005-12-05 19:15:57.000

 Dataset: BGL
- Total registros: 1995
- Distribución por nivel:
Level
Informativo    1596
Error           391
Advertencia       8
Name: count, dtype: int64
- Longitud promedio del mensaje: 4.94 palabras
- Periodo: desde 2005-06-03 22:42:50 hasta 2006-01-03 15:13:09

 Dataset: Hadoop
- Total registros: 1968
- Distribución por nivel:
Level
Informativo    1011
Advertencia     805
Error           152
Name: count, dtype: int64
- Longitud promedio del mensaje: 6.4